In [ ]:
!pip install unstructured[md] sentence-transformers langchain chromadb

In [29]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings,HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter,MarkdownHeaderTextSplitter
from langchain.chains import VectorDBQA
from langchain.document_loaders import TextLoader,DirectoryLoader

  Using cached sentence-transformers-2.2.2.tar.gz (85 kB)
  Preparing metadata (setup.py) ... done
  Using cached torchvision-0.15.2-cp311-cp311-manylinux1_x86_64.whl (6.0 MB)
  Using cached sentencepiece-0.1.99-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.3 MB)
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125925 sha256=56eae50884560b1141e1c19e8f48a91376e82725f64057d6192c0e07d8d0a0a4
  Stored in directory: /home/kglin/.cache/pip/wheels/ff/27/bf/ffba8b318b02d7f691a57084ee154e26ed24d012b0c7805881
Successfully built sentence-transformers


In [48]:
# Load and process the text
loader = DirectoryLoader('./train_dataset_consulting/', glob="*.md")
documents = loader.load()


headers_to_split_on = [
("#", "Header 1"),
("##", "Header 2"),
("###", "Header 3"),
]
text_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on)
texts = text_splitter.split_text(str(documents[0].page_content))
from langchain.text_splitter import RecursiveCharacterTextSplitter

chunk_size = 250
chunk_overlap = 30
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size, chunk_overlap=chunk_overlap
)

# Split
splits = text_splitter.split_documents(texts)
splits

[Document(page_content='Больничные  \nРазмер пособия по временной нетрудоспособности (больничного) зависит от страхового стажа сотрудника:  \nесли страховой стаж от 6 месяцев до 5 лет, то ему оплачивается 60% среднего заработка;  \nот 5 до 8 лет — 80%;  \nболее 8 лет — 100%.'),
 Document(page_content='более 8 лет — 100%.  \nесли стаж сотрудника  менее 6 месяцев, то пособие по временной нетрудоспособности выплачивается в размере, не превышающем МРОТ за полный календарный месяц. С 1 января 2022 года МРОТ составляет 15 279 руб.\nВажно!'),
 Document(page_content='Важно!  \nПосле прохождения испытательного срока, в случае выполнения рабочих задач в период больничного, может быть произведена доплата до средней ЗП, в независимости от страхового стажа.'),
 Document(page_content='(В этом случае необходимо обратиться к своему непосредственному лид-линку и специалисту HR)  \nАдаптация сотрудника  \nПолучи доступы'),
 Document(page_content='Получи доступы  \nТвои доступы ко всем сервисам, которыми

In [49]:
persist_directory = 'db'


model_name = "distiluse-base-multilingual-cased-v1"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': True}

hf = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

vectordb = Chroma.from_documents(documents=splits, embedding=hf, persist_directory=persist_directory)


In [50]:
vectordb.persist()
vectordb = None
vectordb = Chroma(persist_directory=persist_directory, embedding_function=hf)

In [51]:
result = vectordb.similarity_search("Как вести себя на кухне?")
print(result)

[Document(page_content='Правила кухни и офиса  \nПролил что-то или просыпал - протри/подмети, пожалуйста (тряпка/веник и совок в сан.узле около кухни)  \nУвидел забытую грязную кружку/тарелку друга на кухонном столе - убери в посудомойку ;-)'), Document(page_content='Правила кухни и офиса  \nПролил что-то или просыпал - протри/подмети, пожалуйста (тряпка/веник и совок в сан.узле около кухни)  \nУвидел забытую грязную кружку/тарелку друга на кухонном столе - убери в посудомойку ;-)'), Document(page_content='- А что у нас с посудой? Мыть или не мыть, вот в чем вопрос'), Document(page_content='- А что у нас с посудой? Мыть или не мыть, вот в чем вопрос')]


In [46]:
len(result)

4